In [ ]:
# ! pip install phasespace vector awkward matplotlib

A simple example
--------------------------


With these considerations in mind, one can build a decay chain by using the ``set_children`` method of the ``GenParticle``
class. As an example, to build the $B^{+}\to K^+ \pi^+ \pi^-$ decay, we would write:


In [ ]:
from phasespace import GenParticle

BPLUS_MASS = 5.27925
PION_MASS  = 0.13957
KAON_MASS  = 0.49368

piplus = GenParticle('pi+', PION_MASS)
piminus = GenParticle('pi-', PION_MASS)
kplus = GenParticle('K+', KAON_MASS)
bplus = GenParticle('B+', BPLUS_MASS)

bplus.set_children(kplus, piplus, piminus)



Phasespace events can be generated using the ``generate`` method, which gets the number of events to generate as input.
The method returns:

- The normalized weights of each event, as an array of dimension `(n_events,)`.
- The 4-momenta of the generated particles as values of a dictionary with the particle name as key. These momenta
  are expressed as arrays of dimension `(n_events, 4)`.


In [ ]:
N_EVENTS = 1000

weights, particles = bplus.generate(n_events=N_EVENTS)

In [ ]:
particles

For convenience, we write a simple function to extract the 4-momenta, using the [vector](https://vector.readthedocs.io/en/latest/) package:

In [ ]:
import vector

def get_fourmomenta(momenta):
    """slightly more involved with vector.zip; much faster"""
    momenta = momenta.numpy()
    return vector.zip({
        "px": momenta[:, 0],
        "py": momenta[:, 1],
        "pz": momenta[:, 2],
        "E": momenta[:, 3]
    })

pk = get_fourmomenta(particles['K+'])

We can now plot kinematic variables, such as the momentum magnitude of the Kaon. Note that we need to include the **weights** of our MC samples.

In [ ]:
import matplotlib.pyplot as plt

plt.hist(pk.p, weights=weights)